In [1]:
# Basic setup 
!pip -q install langchain huggingface_hub tiktoken
!pip -q install --upgrade together

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [8]:
# Verify Langchain version
!pip show langchain

Name: langchain
Version: 0.0.270
Summary: Building applications with LLMs through composability
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: /opt/python/3.10.8/lib/python3.10/site-packages
Requires: aiohttp, async-timeout, dataclasses-json, langsmith, numexpr, numpy, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 


In [ ]:
# write key to environmental variable
import os

os.environ['OPENAI_API_KEY'] = openai_api_key
os.environ['HUGGINGFACEHUB_API_TOKEN'] = hugging_face_api_key
os.environ['GOOGLE_API_KEY'] = google_api_key
os.environ['TOGETHER_API_KEY'] = together_api_key